## 記得切成Python 3.13.7版下載

In [ ]:
from YouTubeDownload import YouTubeDownload
from MusicSeparation import MusicSeparation

url = input()

audio = YouTubeDownload.download_youtube_audio(url, mode="playlist")
audio


In [ ]:
temp = []
for str1 in audio:
    temp.append(MusicSeparation.run_separation(str1))
temp

## 降噪 Python3.13.7

In [5]:
import os
import torch
import torchaudio

def recursive_vad_clean_overwrite(root_folder):
    """
    遞迴讀取指定資料夾 (包含子資料夾) 內的所有音訊檔案。
    使用 Silero VAD 去除背景雜音 + 殺死孤島 (短雜訊)。
    處理完成後會「覆蓋」原始檔案。
    """
    
    VALID_EXTS = ('.wav', '.mp3', '.flac')

    # 1. 載入模型 (只載入一次，節省時間)
    print(f"正在載入 Silero VAD 模型...")
    try:
        model, utils = torch.hub.load(repo_or_dir='snakers4/silero-vad', 
                                      model='silero_vad', 
                                      force_reload=False, 
                                      onnx=False)
        (get_speech_timestamps, save_audio, read_audio, VADIterator, collect_chunks) = utils
    except Exception as e:
        print(f"模型載入失敗: {e}")
        return

    # 2. 先計算總檔案數 (為了顯示進度條)
    print("正在掃描檔案清單...")
    all_files = []
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            if file.lower().endswith(VALID_EXTS) and not file.startswith("temp_"):
                all_files.append(os.path.join(root, file))
    
    total_files = len(all_files)
    print(f"共找到 {total_files} 個音訊檔案 (含子資料夾)。")

    # 3. 開始遍歷處理
    for idx, file_path in enumerate(all_files):
        # 取得檔名與所在目錄
        dir_path, filename = os.path.split(file_path)
        print(f"[{idx+1}/{total_files}] 處理中: {filename}")
        
        try:
            # --- VAD 核心邏輯開始 ---
            wav = read_audio(file_path, sampling_rate=16000)
            
            raw_timestamps = get_speech_timestamps(
                wav, 
                model, 
                threshold=0.4,
                sampling_rate=16000, 
                min_speech_duration_ms=100, 
                min_silence_duration_ms=100, 
                speech_pad_ms=50
            )

            # 清洗時間戳 (殺死孤島 < 400ms)
            clean_timestamps = []
            removed_count = 0
            for stamp in raw_timestamps:
                duration_ms = (stamp['end'] - stamp['start']) / 16000 * 1000
                if duration_ms < 400: 
                    removed_count += 1
                    continue
                clean_timestamps.append(stamp)

            if removed_count > 0:
                print(f"   └─ 已移除 {removed_count} 個短雜訊孤島")

            # --- 重組與寫入 ---
            if len(clean_timestamps) > 0:
                vad_audio = torch.zeros_like(wav) # 建立全靜音底板
                
                for stamp in clean_timestamps:
                    vad_audio[stamp['start']:stamp['end']] = wav[stamp['start']:stamp['end']]
                
                # 寫入暫存檔 (存放在該檔案原本的資料夾內)
                temp_path = os.path.join(dir_path, f"temp_{filename}")
                
                # 儲存
                save_audio(temp_path, vad_audio, sampling_rate=16000)
                
                # 覆蓋原檔
                if os.path.exists(file_path):
                    os.remove(file_path)
                os.rename(temp_path, file_path)
                
                print(f"   └─ 完成 (已覆蓋)")
            else:
                print("   警告: 檔案全被視為雜訊，保留原檔不動。")

        except Exception as e:
            print(f"   X 錯誤: {file_path} - {e}")
            # 若有暫存檔殘留，嘗試清理
            temp_path = os.path.join(dir_path, f"temp_{filename}")
            if os.path.exists(temp_path):
                os.remove(temp_path)

    print("--- 所有子資料夾處理完畢 ---")

#---使用範例---
target_folder = r"H:\NFU\Featured_Demo\score\訓練模型區塊\song_dataset_台"
recursive_vad_clean_overwrite(target_folder)

正在載入 Silero VAD 模型...
正在掃描檔案清單...
共找到 25 個音訊檔案 (含子資料夾)。
[1/25] 處理中: Lethal Lover-vocals.wav


Using cache found in C:\Users\a0987/.cache\torch\hub\snakers4_silero-vad_master


   └─ 已移除 2 個短雜訊孤島
   └─ 完成 (已覆蓋)
[2/25] 處理中: [音樂] 茄子蛋 浪流連 歌詞-vocals.wav
   └─ 已移除 2 個短雜訊孤島
   └─ 完成 (已覆蓋)
[3/25] 處理中: 你不愛我 Ft. 洋蔥他爸-vocals.wav
   └─ 已移除 3 個短雜訊孤島
   └─ 完成 (已覆蓋)
[4/25] 處理中: 曾瑋中『必巡』／Official Music Video／收錄在曾瑋中『總會有一工』專輯-vocals.wav
   └─ 已移除 1 個短雜訊孤島
   └─ 完成 (已覆蓋)
[5/25] 處理中: 李千娜 Nana Lee – 心花開 (Official Music Video)-vocals.wav
   └─ 已移除 2 個短雜訊孤島
   └─ 完成 (已覆蓋)
[6/25] 處理中: 江蕙 - 家後-vocals.wav
   └─ 完成 (已覆蓋)
[7/25] 處理中: 江蕙 - 落雨聲-vocals.wav
   └─ 完成 (已覆蓋)
[8/25] 處理中: 江蕙 -夢中的情話 MENG CHUNG DE CHING HUA(Official Music Video)-vocals.wav
   └─ 已移除 2 個短雜訊孤島
   └─ 完成 (已覆蓋)
[9/25] 處理中: 江蕙 Jody Chiang & 施文彬 Michael Shih - 傷心酒店 (官方完整KARAOKE版MV)-vocals.wav
   └─ 已移除 1 個短雜訊孤島
   └─ 完成 (已覆蓋)
[10/25] 處理中: 滅火器 Fire EX. - 長途夜車 Southbound Night Bus（滅火器 ╳ 楊雅喆導演）-vocals.wav
   └─ 已移除 1 個短雜訊孤島
   └─ 完成 (已覆蓋)
[11/25] 處理中: 王識賢&孫淑媚《雲中月圓》官方MV-vocals.wav
   └─ 已移除 1 個短雜訊孤島
   └─ 完成 (已覆蓋)
[12/25] 處理中: 珂拉琪 Collage／萬千花蕊慈母悲哀  Mercy Mercy, Green Tara of Thousand Flowers-vocals.wav
   └─ 已移除 1 個短雜訊孤島
   

## 記得切成Python3.9.13再跑對齊

In [11]:
import os
import csv
import glob
import json
import shutil
import tempfile
import subprocess
import sys
from pydub import AudioSegment

def align_song_aeneas(audio_path, output_dir):
 """
 Aeneas對齊歌曲+歌詞,用Pydub切割成一句音檔+CSV輸出
 """
 os.makedirs(output_dir, exist_ok=True)
 base_name = os.path.splitext(os.path.basename(audio_path))[0]

 #嘗試找同名txt歌詞檔
 lyrics_txt = os.path.join(os.path.dirname(audio_path), f"{base_name}.txt")
 
 if not os.path.exists(lyrics_txt):
  print(f"找不到歌詞檔{lyrics_txt},Aeneas將無法對齊此歌曲。")
  return None
 
 HARDCODED_TEMP_DIR = r"C:\aeneas_temp_workdir"
 os.makedirs(HARDCODED_TEMP_DIR, exist_ok=True)
 
 temp_audio_path = os.path.join(HARDCODED_TEMP_DIR, "temp_audio.wav")
 temp_text_path = os.path.join(HARDCODED_TEMP_DIR, "temp_lyrics.txt")
 
 json_path = os.path.join(output_dir, "aligned.json")
 
 try:
  #複製/轉換檔案到臨時位置
  try:
   audio_for_temp = AudioSegment.from_file(audio_path)
   audio_for_temp.export(temp_audio_path, format="wav")
  except Exception as e:
   print(f"讀取/複製音訊檔{base_name}時(Pydub)發生錯誤: {e}")
   return None
   
  shutil.copy(lyrics_txt, temp_text_path)

  config_string = (
            "task_language=cmn|"     #語言要改(中)
            #"task_language=en|"     #語言要改(英)
            #"task_language=zh-min-nan|"  #語言要改(台)
            #"task_language=nan|"  #語言要改(台)
            "is_text_type=plain|"
            "os_task_file_format=json|"
            "tts=espeak|"                     # 指定使用 espeak 驅動
            "tts_path=espeak-ng"              # 指定執行檔名稱為 espeak-ng
        )

    # 建立命令列
  command = [
        sys.executable,
        "-m", "aeneas.tools.execute_task",
        temp_audio_path,
        temp_text_path,
        config_string,                    # 放入上面定義好的設定字串
        os.path.abspath(json_path)
    ]

  print(f"🎤 Working on: {base_name}")
  print(f"(Aeneas 正在對齊 {os.path.basename(audio_path)} ...)")

  #執行命令列
  result = subprocess.run(
   command, 
   capture_output=True, 
   text=True, 
   encoding="utf-8"
  )

  #檢查命令是否成功
  if result.returncode != 0:
   print(f"Aeneas對齊失敗 {base_name}。")
   print("--- Aeneas錯誤輸出 ---")
   print(result.stdout)
   print(result.stderr)
   print("---------------------------")
   return None

 except Exception as e:
  print(f"Aeneas發生未預期的錯誤(subprocess): {e}")
  return None
  
 finally:
  #刪除臨時檔案
  if os.path.exists(temp_audio_path):
   os.remove(temp_audio_path)
  if os.path.exists(temp_text_path):
   os.remove(temp_text_path)
 
 print(f"(Aeneas 對齊完成，正在切割音檔...)")

 #讀取輸出的JSON並切割
 try:
  audio = AudioSegment.from_file(audio_path)
  
  with open(json_path, 'r', encoding='utf-8') as f:
   align_data = json.load(f)

  csv_path = os.path.join(output_dir, "metadata.csv")
  with open(csv_path, "w", newline="", encoding="utf-8") as f:
   writer = csv.writer(f)
   writer.writerow(["file", "text"]) 

   fragments = align_data.get("fragments")
   if not fragments:
    print(f"對齊的JSON{json_path}中沒有找到'fragments'。")
    return None

   for i, fragment in enumerate(fragments):
    text = fragment.get("lines", [""])[0].strip()
    if not text:
     continue 

    begin_ms = float(fragment["begin"]) * 1000
    end_ms = float(fragment["end"]) * 1000

    segment = audio[begin_ms:end_ms]

    out_wav = os.path.join(output_dir, f"line_{i:04d}.wav")
    segment.export(
     out_wav, 
     format="wav", 
     parameters=["-ar", "16000", "-ac", "1"]
    )
    
    writer.writerow([out_wav, text])

  print(f"(切割完成，已儲存至 {output_dir})")
  return csv_path

 except Exception as e:
  print(f"Pydub切割或讀取JSON時發生錯誤: {e}")
  return None

def process_folder_aeneas(input_dir, output_root="processed_songs_aeneas", merged_csv="metadata_all.csv"):
    #處理資料夾內所有歌曲
    os.makedirs(output_root, exist_ok=True)
    all_rows = []

    audio_files = []
    for ext in ("*.wav", "*.mp3", "*.flac"):
        audio_files.extend(glob.glob(os.path.join(input_dir, ext)))

    if not audio_files:
        print(f"在{input_dir}中找不到任何音訊檔案。")
        return

    print(f"總共找到 {len(audio_files)} 個音訊檔案，開始處理...")
    
    total_skipped = 0 #追蹤過濾掉的資料數

    for audio_path in audio_files:
        base_name = os.path.splitext(os.path.basename(audio_path))[0]
        song_output = os.path.join(output_root, base_name)
        
        csv_path = align_song_aeneas(audio_path, song_output)

        if csv_path:
            with open(csv_path, "r", encoding="utf-8") as f:
                reader = csv.reader(f)
                next(reader)  #跳過標頭
                
                for row in reader:
                    #取得原始資料
                    file_path = row[0]
                    raw_text = row[1]
                    
                    #移除句首句尾的空白
                    cleaned_text = raw_text.strip()
                    
                    #移除所有的空格
                    cleaned_text = cleaned_text.replace(" ", "")   #語言是英文的話要註解掉
                    cleaned_text = cleaned_text.replace("?", "")
                    cleaned_text = cleaned_text.replace("(", "")
                    cleaned_text = cleaned_text.replace(")", "")
                    cleaned_text = cleaned_text.replace(".", "")
                    cleaned_text = cleaned_text.replace("。", "")
                    cleaned_text = cleaned_text.replace("，", "")
                    if not cleaned_text:
                        #如果清理後是空的就跳過
                        total_skipped += 1
                        continue
                        
                    relative_path = os.path.relpath(file_path, output_root)
                    
                    formatted_path = relative_path.replace(os.path.sep, "/")
                    
                    all_rows.append([formatted_path, cleaned_text])
            
    merged_path = os.path.join(output_root, merged_csv)
    if not all_rows:
        print("\n所有歌曲都處理失敗,沒有產生任何 metadata。")
        return
        
    with open(merged_path, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["file", "text"])
        writer.writerows(all_rows)

    print(f"\n所有歌曲已處理完成!")
    print(f"輸出資料夾: {output_root}")
    print(f"合併metadata: {merged_path}(共{len(all_rows)}筆資料)")
    if total_skipped > 0:
        print(f"ℹ已過濾掉{total_skipped}筆空白/靜音資料。")

if __name__ == "__main__":
 INPUT_FOLDER = "song_dataset_台" 
 
 OUTPUT_FOLDER = "processed_songs_台"

 process_folder_aeneas(INPUT_FOLDER, OUTPUT_FOLDER)

總共找到 25 個音訊檔案，開始處理...
🎤 Working on: Lethal Lover-vocals
(Aeneas 正在對齊 Lethal Lover-vocals.wav ...)
(Aeneas 對齊完成，正在切割音檔...)
(切割完成，已儲存至 processed_songs_台\Lethal Lover-vocals)
🎤 Working on: [音樂] 茄子蛋 浪流連 歌詞-vocals
(Aeneas 正在對齊 [音樂] 茄子蛋 浪流連 歌詞-vocals.wav ...)
(Aeneas 對齊完成，正在切割音檔...)
(切割完成，已儲存至 processed_songs_台\[音樂] 茄子蛋 浪流連 歌詞-vocals)
🎤 Working on: 你不愛我 Ft. 洋蔥他爸-vocals
(Aeneas 正在對齊 你不愛我 Ft. 洋蔥他爸-vocals.wav ...)
(Aeneas 對齊完成，正在切割音檔...)
(切割完成，已儲存至 processed_songs_台\你不愛我 Ft. 洋蔥他爸-vocals)
🎤 Working on: 曾瑋中『必巡』／Official Music Video／收錄在曾瑋中『總會有一工』專輯-vocals
(Aeneas 正在對齊 曾瑋中『必巡』／Official Music Video／收錄在曾瑋中『總會有一工』專輯-vocals.wav ...)
(Aeneas 對齊完成，正在切割音檔...)
(切割完成，已儲存至 processed_songs_台\曾瑋中『必巡』／Official Music Video／收錄在曾瑋中『總會有一工』專輯-vocals)
🎤 Working on: 李千娜 Nana Lee – 心花開 (Official Music Video)-vocals
(Aeneas 正在對齊 李千娜 Nana Lee – 心花開 (Official Music Video)-vocals.wav ...)
(Aeneas 對齊完成，正在切割音檔...)
(切割完成，已儲存至 processed_songs_台\李千娜 Nana Lee – 心花開 (Official Music Video)-vocals)
🎤 Working on: 江蕙 - 家後-vocals
(Aen

## 正確率評分 切回3.13.7

In [ ]:
import Levenshtein
import difflib

#正規化函數，把 "妳"、"祢" 視為 "你"，再視為在
def normalize_text(text: str) -> str:
    mapping = {
        "妳": "你",
        "祢": "你",
        "再": "在",
    }
    return "".join(mapping.get(ch, ch) for ch in text)

def calc_accuracy_lyrics(gt_file, test_file, report_file="compare_report.txt"):
    #讀取檔案
    with open(gt_file, "r", encoding="utf-8") as f:
        gt_text = f.read()
    with open(test_file, "r", encoding="utf-8") as f:
        test_text = f.read()

    #移除空白與換行，並正規化
    gt_text = normalize_text(gt_text.replace(" ", "").replace("\n", ""))
    test_text = normalize_text(test_text.replace(" ", "").replace("\n", ""))

    #計算編輯距離
    distance = Levenshtein.distance(gt_text, test_text)
    max_len = max(len(gt_text), len(test_text))
    accuracy = (max_len - distance) / max_len if max_len > 0 else 0

    #用difflib做逐字比對
    diff = difflib.SequenceMatcher(None, gt_text, test_text)
    report_parts = []
    for tag, i1, i2, j1, j2 in diff.get_opcodes():
        if tag == "equal":
            report_parts.append(gt_text[i1:i2])  
        elif tag == "replace":
            for a, b in zip(gt_text[i1:i2], test_text[j1:j2]):
                report_parts.append(f"【預測:{b}/正確:{a}】")
            if (i2 - i1) > (j2 - j1):
                for a in gt_text[j2 - j1 + i1:i2]:
                    report_parts.append(f"【缺少:{a}】")
            elif (j2 - j1) > (i2 - i1):
                for b in test_text[i2 - i1 + j1:j2]:
                    report_parts.append(f"【多餘:{b}】")
        elif tag == "delete":
            for a in gt_text[i1:i2]:
                report_parts.append(f"【缺少:{a}】")
        elif tag == "insert":
            for b in test_text[j1:j2]:
                report_parts.append(f"【多餘:{b}】")
        report_parts.append('\n')

    result_text = []
    result_text.append(f"總字數: {max_len}")
    result_text.append(f"最少編輯次數: {distance}")
    result_text.append(f"正確率: {accuracy*100:.2f}%")
    result_text.append("\n=== 詳細比對結果===")
    result_text.append("".join(report_parts))

    print("\n".join(result_text))

    with open(report_file, "w", encoding="utf-8") as f:
        f.write("\n".join(result_text))
    print(f"\n比對結果已輸出至 {report_file}")

In [ ]:
#呼叫方法(正確歌詞,測試歌詞)
calc_accuracy_lyrics("挪威的森林_歌詞.txt", "挪威的森林_1120_微調58首.txt")